 ## Hello to everyone ! this is our first kaggle competition test.
 
In this notebook we are testing our baseline solution. We trained 18 models (1 model for each label,models are based on efficient net architecture) on selection of monolabel dataset. Training datasets were data augmented and equilibrated. In this notebook we are making predictions using loop of 18 binary models.

We will appreciate every critical comment or remark! 

Thank you!

# Install Libraries

In [ ]:
!pip install "../input/keras-application/Keras_Applications-1.0.8-py3-none-any.whl"
!pip install "../input/efficientnet111/efficientnet-1.1.1-py3-none-any.whl"
!pip install "../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl"
!pip install "../input/hpapytorchzoozip/pytorch_zoo-master"
!pip install "../input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"
#!pip install "../input/tfexplainforoffline/tf_explain-0.2.1-py3-none-any.whl"

In [ ]:
import os, glob
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

from pycocotools import _mask as coco_mask
import random
import base64
import numpy as np
import typing as t
import zlib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
import hpacellseg.cellsegmentator as cellsegmentator
from hpacellseg.utils import label_cell, label_nuclei
import cv2

from keras.optimizers import Adam
from keras.callbacks import Callback
from efficientnet.keras import EfficientNetB0
from keras.layers import Dense, Flatten
from keras.models import Model, load_model
from albumentations import Compose, VerticalFlip, HorizontalFlip, Rotate, GridDistortion
from IPython.display import Image, display
import gc
from tqdm.auto import tqdm
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt 
from PIL import Image

%matplotlib inline

print("Done--------")

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    tf.compat.v1.disable_eager_execution()

# DEBUT

In [ ]:
# DEBUT
TEST_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/test/'
TRAIN_IMGS_FOLDER = '../input/hpa-single-cell-image-classification/train/'
IMG_HEIGHT = IMG_WIDTH = 128
MODELS_folder="../input/single-labelmodelsaved/"

In [ ]:
import os, glob
#Generer le dataframe avec les ID, taille des images
# Capture all the relevant full image paths for the competition dataset
TEST_IMG_PATHS = sorted([os.path.join(TEST_IMGS_FOLDER, f_name) for f_name in os.listdir(TEST_IMGS_FOLDER)])
print(f"\n... Recall that 4 training images compose one example (R,G,B,Y) ...")
print(f"... \t– i.e. The first 4 training files are:")
for path in [x.rsplit('/',1)[1] for x in TEST_IMG_PATHS[:4]]: print(f"... \t\t– {path}")
print(f"\n... The number of training images is {len(TEST_IMG_PATHS)} i.e. {len(TEST_IMG_PATHS)//4} 4-channel images ...")


In [ ]:
#liste des ID
id_test=[]
for i in TEST_IMG_PATHS:
    id=i.split("/")
    id=id[-1]
    id=id.split("_")
    id=id[0]
    if id not in id_test:
        id_test.append(id)



In [ ]:

#Creer le dataset Submission
submission_dataset=pd.DataFrame(columns=['ID', 'ImageWidth',"ImageHeight"])

submission_dataset['ID']=id_test
submission_dataset

I will try to look if submission is working only with 3 images.

In [ ]:
submission_dataset=submission_dataset.sample(3)

In [ ]:

submission_dataset['ImageWidth']=submission_dataset['ID'].apply(lambda x:(np.array(Image.open(TEST_IMGS_FOLDER + x + '_red.png'))).shape[0])

In [ ]:
submission_dataset['ImageHeight']=submission_dataset['ID'].apply(lambda x:(np.array(Image.open(TEST_IMGS_FOLDER + x + '_red.png'))).shape[1])

In [ ]:
def open_rgby_test(image_id): #a function that reads RGBY image with resizing
    colors = ['red','green','blue','yellow']
    img = [cv2.imread(os.path.join(TEST_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE) for color in colors]
    img = np.stack(img, axis=-1)
    #img_resized = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    #img_resized=img_resized/255
    return img

In [ ]:
def open_rgby(image_id): #a function that reads RGBY image with resizing
    colors = ['red','green','blue','yellow']
    img = [cv2.imread(os.path.join(TEST_IMGS_FOLDER, f'{image_id}_{color}.png'), cv2.IMREAD_GRAYSCALE) for color in colors]
    img = np.stack(img, axis=-1)
    img_resized = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH))
    img_resized=img_resized/255
    return img_resized

In [ ]:
def get_masks(img, test=True):
    try:
        print("trying--------")      
        images = [[img[:, :, 0] ],
                  [img[:, :, 3] ],
                  [img[:, :, 2] ]]
        print("trying done--------")      
        nuc_segmentations = segmentator.pred_nuclei(images[2])
        cell_segmentations = segmentator.pred_cells(images)
        cell_masks = []
        for i in tqdm(range(len(cell_segmentations)), desc='Labeling cells..'):
            _, cell_mask = label_cell(nuc_segmentations[i], cell_segmentations[i])
            cell_masks.append(cell_mask)
        return cell_masks
    except:
        raise ValueError('Segmentation failed')


In [ ]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
  """Converts a binary mask into OID challenge encoding ascii text."""

  # check input mask --
  if mask.dtype != np.bool:
    raise ValueError(
        "encode_binary_mask expects a binary mask, received dtype == %s" %
        mask.dtype)

  mask = np.squeeze(mask)
  if len(mask.shape) != 2:
    raise ValueError(
        "encode_binary_mask expects a 2d mask, received shape == %s" %
        mask.shape)

  # convert input mask to expected COCO API input --
  mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
  mask_to_encode = mask_to_encode.astype(np.uint8)
  mask_to_encode = np.asfortranarray(mask_to_encode)

  # RLE encode mask --
  encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

  # compress and base64 encoding --
  binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
  base64_str = base64.b64encode(binary_str)
  return base64_str.decode('ascii')

In [ ]:
# GENERE LE DF DE MASK

In [ ]:
#submission_dataset=submission_dataset.loc[0:3,]
#submission_dataset.head()

In [ ]:
# instanciation du model de segmentation en réutilisant les poids préentrainés
NUC_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '../input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device='cuda',
    padding=True,
    multi_channel_model=True
)


In [ ]:
submission_dataset['MASK']=submission_dataset['ID'].apply(lambda x:get_masks(open_rgby_test(str(x))))

In [ ]:
submission_dataset['MASK_BINAIRE']=submission_dataset['MASK'].apply(lambda x:x[0]>0)

In [ ]:
submission_dataset['MASK_BINAIRE_ENCODE']=submission_dataset['MASK_BINAIRE'].apply(lambda x:encode_binary_mask(x))


In [ ]:
#GENERE LA PREDICTION

In [ ]:
model=[]
N=7000
labels=np.arange(18)
labels

for label in tqdm(labels):    
    model.append(load_model("../input/single-labelmodelsaved/model_label_{}_monolabel_{}.h5".format(label,N)))
    submission_dataset['prediction_{}'.format(label)]=submission_dataset['ID'].apply(lambda x: model[label].predict(np.expand_dims((open_rgby(str(x))),0),use_multiprocessing=True))
    submission_dataset['prediction_result_{}'.format(label)]=submission_dataset['prediction_{}'.format(label)].apply(lambda x: 1 if x>0.5 else 0) 
    
#sub_dataset[df].to_csv('./sub_dataset_{}_{}'.format(label,df))

In [ ]:
# POUR LABEL 18
for index,row in submission_dataset.iterrows():
    total=0
    for j in range (18):
        colonne="prediction_result_{}".format(j)
        total=total + row[colonne]
    if total==0:
        submission_dataset.loc[index,'prediction_result_18']=1
    else:
        submission_dataset.loc[index,'prediction_result_18']=0

In [ ]:
# GENERE LA PREDICTION STRING

In [ ]:
liste_prediction=["prediction_result_{}".format(i) for i in range(19)]
liste_label=[i for i in range(19)]
submission_dataset['PredictionString']=""

#pour genérer la liste
for index,row in submission_dataset.iterrows():
    k=0
    predstring=''
    for i in liste_prediction:
        pred=row[i]
        if pred==1:
            enc=row['MASK_BINAIRE_ENCODE'] 
            label=liste_label[k]
            predstring = predstring + str(label) + ' 1.0 ' + enc + ' '
            k=k+1
    #print(index, predstring)
    submission_dataset.loc[index,'PredictionString']=predstring

In [ ]:
#GENERE LE submission csv

In [ ]:
submission_sample=submission_dataset.loc[:,['ID','ImageWidth','ImageHeight','PredictionString']]
submission_sample.head()

In [ ]:
#submission_sample.rename(columns={'ImageHeight_x': 'ImageHeight','ImageWidth_x':'ImageWidth','SubmissionString':'PredictionString'}, inplace=True)

In [ ]:
#submission_sample.loc[225,'PredictionString'].split(' ')[5]

In [ ]:
submission_sample.to_csv('submission.csv',index=False)

In [ ]:
#test=pd.read_csv('./submission.csv')
#test.head()